In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install roboflow

from roboflow import Roboflow
import os

# Roboflow API 키 설정
rf = Roboflow(api_key="qthwFyiSbLdwOFVygVhW")

# 프로젝트와 데이터셋 버전 지정
project = rf.workspace("tata-meta-19tsj").project("people-counting-naew1")
version = project.version(2)
# https://universe.roboflow.com/keio-dba-team/crowdhuman-nur7g/dataset/3# 21990 이미지로 첫번째 훈련
# 데이터셋 다운로드
dataset = version.download("yolov8")

# 다운로드된 데이터셋의 경로 출력
print("Dataset downloaded to:", dataset)

# 데이터셋 내용 확인
print("\nContents of the dataset directory:")
!ls {dataset}

In [ ]:
!pip install PyYAML #파이썬에서 YAML 파일을 사용하기 위해서 PyYAML 라이브러리 설치

In [ ]:
import yaml

data = {
    'train': '/content/Human/train/images',
    'val': '/content/Human/valid/images',
    'test': '/content/Human/test/images',
    'nc': 2,  # 클래스 수
    'names': ['head', 'person']  # 클래스 이름
}

os.makedirs('/content/Human', exist_ok=True)
yaml_path = '/content/Human/Human_data.yaml'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print(f"YAML 파일이 생성되었습니다: {yaml_path}")
print("내용:")
print(yaml.dump(data, default_flow_style=False))

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

In [ ]:
# Colab 노트북 초기화 코드
import torch
print("CUDA available:", torch.cuda.is_available())

# 데이터 경로 및 학습 설정
data_path = '/content/Human/Human_data.yaml'
epochs = 100
patience = 30
batch_size = 32
img_size = 416

# 모델 학습
model.train(data=data_path, epochs=epochs, patience=patience, batch=batch_size, imgsz=img_size)

# 학습 종료 후 GPU 메모리 해제
torch.cuda.empty_cache()

In [ ]:
# Colab 노트북 초기화 코드
import torch
print("CUDA available:", torch.cuda.is_available())


# 기존 체크포인트 로드 (경로를 실제 체크포인트 파일로 변경)
checkpoint_path = '/content/runs/detect/train5/weights/best.pt'  # 또는 'last.pt'

# 모델 로드
model = YOLO(checkpoint_path)

# 데이터 경로 및 추가 학습 설정
data_path = '/content/Human/Human_data.yaml'
additional_epochs = 20  # 추가로 학습할 에포크 수
patience = 30
batch_size = 32
img_size = 416

# 모델 추가 학습
model.train(data=data_path, epochs=additional_epochs, patience=patience, batch=batch_size, imgsz=img_size)

# 학습 종료 후 GPU 메모리 해제
torch.cuda.empty_cache()

# 결과 파일 경로 확인
import os

# 기본적으로 runs/train 디렉토리에 저장됨
train_dir = 'runs/train1'
exp_folders = sorted(os.listdir(train_dir), key=lambda x: os.path.getctime(os.path.join(train_dir, x)))
latest_exp = exp_folders[-1]  # 가장 최근 실행 폴더

print(f"학습 결과는 다음 경로에 저장되었습니다: {train_dir}/{latest_exp}")


In [ ]:
# 모델 평가
from ultralytics import YOLO

# 학습된 모델 로드
model = YOLO('/content/drive/MyDrive/runs/detect/train/weights/best.pt')  # 학습된 모델 파일의 경로로 변경해주세요

# 데이터셋 경로 (검증 데이터셋이 포함된 YAML 파일)
data_path = '/content/drive/MyDrive/Human/Human_data.yaml'

# 모델 평가 실행
results = model.val(data=data_path)

# 평가 결과 출력
print("\nValidation Results:")
print(f"mAP50 (Box): {results.box.map50:.4f}")
print(f"mAP50-95 (Box): {results.box.map:.4f}")
print(f"Precision: {results.box.mp:.4f}")
print(f"Recall: {results.box.mr:.4f}")

# 추가적인 메트릭 (필요시 주석 해제)
print(f"Speed: {results.speed['inference']:.2f} ms pre-process, {results.speed['inference']:.2f} ms inference, {results.speed['postprocess']:.2f} ms post-process per image")

In [ ]:
reuslt = model.predict(source = '/content/Human/test/images', save = True)

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import time
import csv
from datetime import datetime

import math

class CrowdGuardAlert:
    def __init__(self, model_path, area_size):
        self.model = YOLO(model_path)
        self.area_size = area_size

    def calculate_average_distance(self, density):
        # 밀도를 기반으로 평균 거리 계산 (단순화된 모델)
        return math.sqrt(1 / density)

    def generate_alert(self, density):
        avg_distance = self.calculate_average_distance(density)
        
        if avg_distance < 0.3:
            return "극도의 위험! 즉시 대피하세요! 압사 위험이 매우 높습니다.", 4
        elif avg_distance < 0.5:
            return "심각한 위험! 군중 밀집도가 매우 높습니다. 즉시 주의하세요!", 3
        elif avg_distance < 0.7:
            return "주의! 군중이 밀집되고 있습니다. 이동에 주의하세요.", 2
        elif avg_distance < 1.0:
            return "경계! 군중이 모이고 있습니다. 상황을 주시하세요.", 1
        else:
            return "안전: 현재 군중 밀도가 안전한 수준입니다.", 0

    def send_emergency_message(self, message):
        # 이동통신사 API 연동 예시 (실제 구현 필요)
        print(f"긴급 재난 문자: {message}")
        # API 호출 예시:
        # response = requests.post('https://emergency-api.example.com/send',
        #                          json={'message': message, 'level': 'urgent'})
        # if response.status_code == 200:
        #     print("긴급 메시지 전송 성공")
        # else:
        #     print("긴급 메시지 전송 실패")

    def log_data(self, people_count, density, alert):
        with open(self.log_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([datetime.now(), people_count, density, alert])

    def monitor_crowd(self, camera_index=0):
        cap = cv2.VideoCapture(camera_index)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            people_count, density = self.analyze_frame(frame)
            alert = self.generate_alert(density)

            if alert:
                self.send_emergency_message(alert)

            # 데이터 로깅
            self.log_data(people_count, density, alert)

            # 화면에 정보 표시
            cv2.putText(frame, f"People: {people_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Density: {density:.2f}/m^2", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow("CrowdGuard Alert", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

# 사용 예시
if __name__ == "__main__":
    model_path = "path/to/your/trained/model.pt"
    area_size = 18.24  # 이태원 사고 현장 면적 (m^2)
    camera_index = 0  # 노트북 웹캠 사용

    crowd_guard = CrowdGuardAlert(model_path, area_size)
    crowd_guard.monitor_crowd(camera_index)